# Chapter 12 Analysis of Single Factor Experiments

In [25]:
import json

import polars as pl
from polars import col, lit
from scipy import stats
import numpy as np
import altair as alt

# random number generator
RNG = np.random.default_rng()

## Section 12.1

### 1.

In [74]:
with open('Ex12-1.json') as file:
    data = pl.from_dict(json.load(file))
print(data)

shape: (3, 6)
┌──────┬──────────┬─────────┬──────────┬─────────┬──────────┐
│ n    ┆ SugarAvg ┆ SugarSD ┆ FiberAvg ┆ FiberSD ┆ Location │
│ ---  ┆ ---      ┆ ---     ┆ ---      ┆ ---     ┆ ---      │
│ f64  ┆ f64      ┆ f64     ┆ f64      ┆ f64     ┆ str      │
╞══════╪══════════╪═════════╪══════════╪═════════╪══════════╡
│ 20.0 ┆ 4.8      ┆ 2.138   ┆ 1.68     ┆ 1.166   ┆ Shelf1   │
│ 20.0 ┆ 9.85     ┆ 1.985   ┆ 0.95     ┆ 1.162   ┆ Shelf2   │
│ 20.0 ┆ 6.1      ┆ 1.865   ┆ 2.17     ┆ 1.277   ┆ Shelf3   │
└──────┴──────────┴─────────┴──────────┴─────────┴──────────┘


Transform to a long format that is easier to work with in subsequent tasks.

In [75]:
data = (
    data
    .select(
        factor='Location',
        n=col('n').cast(int),
        container=pl.concat_list(
            pl.struct(dataset=lit('sugar'), μ='SugarAvg', s='SugarSD'),
            pl.struct(dataset=lit('fiber'), μ='FiberAvg', s='FiberSD')))
    .explode('container')
    .unnest('container')
    .sort('dataset', 'factor'))
print(data)

shape: (6, 5)
┌────────┬─────┬─────────┬──────┬───────┐
│ factor ┆ n   ┆ dataset ┆ μ    ┆ s     │
│ ---    ┆ --- ┆ ---     ┆ ---  ┆ ---   │
│ str    ┆ i64 ┆ str     ┆ f64  ┆ f64   │
╞════════╪═════╪═════════╪══════╪═══════╡
│ Shelf1 ┆ 20  ┆ fiber   ┆ 1.68 ┆ 1.166 │
│ Shelf2 ┆ 20  ┆ fiber   ┆ 0.95 ┆ 1.162 │
│ Shelf3 ┆ 20  ┆ fiber   ┆ 2.17 ┆ 1.277 │
│ Shelf1 ┆ 20  ┆ sugar   ┆ 4.8  ┆ 2.138 │
│ Shelf2 ┆ 20  ┆ sugar   ┆ 9.85 ┆ 1.985 │
│ Shelf3 ┆ 20  ┆ sugar   ┆ 6.1  ┆ 1.865 │
└────────┴─────┴─────────┴──────┴───────┘


#### (a)

For (a), remember to use the _pooled_ $\hat s$.
$$
\text{CI} = \hat\mu_\text{group} \pm t_{N-a,\ \alpha/2} \frac{\hat{s}_\text{pool}}{\sqrt{n_\text{group}}}
$$

In [76]:
def CI(data: pl.DataFrame, α: float=0.05) -> pl.DataFrame:
    """ 
    returns the studentized CIs for each group in the data using a pooled s.

    data columns:
    - dataset: string id of dataset
    - n: group size
    - μ: group mean
    - s: group standard deviation
    """
    dof = col('n').sum() - pl.len()
    t_crit = dof.map_elements(lambda v: stats.t.ppf(1-α/2, v))
    s_pool = ((col('s')**2 * (col('n') - 1)).sum() / dof).sqrt()
    
    e = t_crit * s_pool / col('n').sqrt()
    return (
        data
        .group_by('dataset')
        .agg(
            'factor',
            CI=pl.concat_list(col('μ') - e, col('μ') + e))
        .explode('factor', 'CI'))
            
print(ans := CI(data))

shape: (6, 3)
┌─────────┬────────┬───────────────────────┐
│ dataset ┆ factor ┆ CI                    │
│ ---     ┆ ---    ┆ ---                   │
│ str     ┆ str    ┆ list[f64]             │
╞═════════╪════════╪═══════════════════════╡
│ sugar   ┆ Shelf1 ┆ [3.904862, 5.695138]  │
│ sugar   ┆ Shelf2 ┆ [8.954862, 10.745138] │
│ sugar   ┆ Shelf3 ┆ [5.204862, 6.995138]  │
│ fiber   ┆ Shelf1 ┆ [1.141407, 2.218593]  │
│ fiber   ┆ Shelf2 ┆ [0.411407, 1.488593]  │
│ fiber   ┆ Shelf3 ┆ [1.631407, 2.708593]  │
└─────────┴────────┴───────────────────────┘


Let's visualize the CIs.

In [77]:
(
    alt.Chart(
        ans.select('dataset', 'factor',
            low=col('CI').list.first(),
            high=col('CI').list.last()))
    .mark_line(strokeWidth=2)
    .encode(
        alt.X('low').title('CI'),
        alt.X2('high'),
        alt.Y('factor').title(None),
        alt.Row('dataset').title(None))
    .resolve_scale(x='independent')
)

alt.Chart(...)

- Sugar: The CIs of Shelf1 and Shelf3 overlap, so no obvious differences. But Shelf2 stands out from the rest.
- Fiber: By the same token, Shelf2 and Shelf3 are different becuuse their CIs do not overlap.

#### (b)

In [81]:
def ANOVA(data: pl.DataFrame) -> pl.DataFrame:
    """ 
    returns the 1-way ANOVA table as a DataFrame.

    assuming the correct conditions of data:
    - normality assumption
    - constant variance

    data columns:
    - n: group size
    - μ: group mean
    - s: group standard deviation
    """
    def item(expr: pl.Expr) -> float:
        return data.select(expr).item()

    μ_total = (col('μ') * col('n')).sum() / col('n').sum()
    
    # factor A sum of squares
    ssa = ((col('μ') - μ_total)**2 * col('n')).sum()
    dof_ssa = pl.len() - 1
    
    # error sum of squares
    sse = (col('s')**2 * (col('n') - 1)).sum()
    dof_sse = col('n').sum() - pl.len()
    
    msa = ssa / dof_ssa # factor A mean square
    mse = sse / dof_sse # error mean square

    return (
        data
        .group_by('dataset')
        .agg(ssa=ssa, dof_ssa=dof_ssa, sse=sse, dof_sse=dof_sse, msa=msa, mse=mse)
        .select(
            'dataset', 
            source=pl.concat_list(lit('factor'), lit('error'), lit('total')),
            SS=pl.concat_list('ssa', 'sse', col('ssa') + col('sse')),
            dof=pl.concat_list('dof_ssa', 'dof_sse', col('dof_ssa') + col('dof_sse')),
            MS=pl.concat_list('msa', 'mse', None),
            F=pl.concat_list(col('msa') / col('mse'), None, None))
        .explode('source', 'SS', 'dof', 'MS', 'F'))


print(ans := ANOVA(data))

shape: (6, 6)
┌─────────┬────────┬────────────┬─────┬────────────┬───────────┐
│ dataset ┆ source ┆ SS         ┆ dof ┆ MS         ┆ F         │
│ ---     ┆ ---    ┆ ---        ┆ --- ┆ ---        ┆ ---       │
│ str     ┆ str    ┆ f64        ┆ i64 ┆ f64        ┆ f64       │
╞═════════╪════════╪════════════╪═════╪════════════╪═══════════╡
│ sugar   ┆ factor ┆ 275.033333 ┆ 2   ┆ 137.516667 ┆ 34.409292 │
│ sugar   ┆ error  ┆ 227.800386 ┆ 57  ┆ 3.996498   ┆ null      │
│ sugar   ┆ total  ┆ 502.833719 ┆ 59  ┆ null       ┆ null      │
│ fiber   ┆ factor ┆ 15.076     ┆ 2   ┆ 7.538      ┆ 5.209964  │
│ fiber   ┆ error  ┆ 82.470051  ┆ 57  ┆ 1.446843   ┆ null      │
│ fiber   ┆ total  ┆ 97.546051  ┆ 59  ┆ null       ┆ null      │
└─────────┴────────┴────────────┴─────┴────────────┴───────────┘


In [11]:
ans.it

34.40929200181425

#### (c)

### 2.

## Section 12.2